In [85]:
import sqlite3
import pandas as pd
import numpy as np
from pandasql import sqldf
from geopy.distance import great_circle
import hashlib
import os
db_path = 'cs.db'  # get the absolute path of the database file
import pickle

In [314]:
class DatabaseHandler:
   
    def get_table(name):
        if name == 'sizes':
            tbl = ['', 'Small', 'Medium', 'Large']
        elif name == 'types':
            tbl = ['', 'Public', 'Private Nonprofit', 'Private For-Profit']
        elif name == 'urban':
            tbl = ['', 'City', 'Suburban', 'Town', 'Rural']
        elif name == 'missions':
            tbl = ['', 'Men-Only College', 'Women-Only College', 'Alaska Native Native Hawaiian Serving Institution', \
                'Asian American Native American Pacific Islander-Serving Institution', \
                'Hispanic-Serving Institution', 'Historically Black College and University', \
                'Native American Non-Tribal Institution', 'Predominantly Black Institution',
                'Tribal College and University']
        elif name == 'states':
            with sqlite3.connect(db_path) as conn:
                tbl = list(pd.read_sql("SELECT DISTINCT STABBR FROM inst ORDER BY 1", conn).STABBR)
                tbl.insert(0, '')
        elif name == 'fields':
            with sqlite3.connect(db_path) as conn:
                tbl = list(pd.read_sql("SELECT DISTINCT CIPDESC FROM fields ORDER BY 1", conn).CIPDESC)
                tbl.insert(0, '')
        elif name == 'religs':
            with sqlite3.connect(db_path) as conn:
                tbl = list(pd.read_sql("SELECT DISTINCT NAME FROM relig ORDER BY 1", conn).NAME)
                tbl.insert(0, '')
        elif name == 'relig':
            with sqlite3.connect(db_path) as conn:
                tbl = pd.read_sql("SELECT * FROM relig", conn)
        elif name == 'inst':
            with sqlite3.connect(db_path) as conn:
                tbl = pd.read_sql("""SELECT UNITID, INSTNM, ZIP, CITY, STABBR, LATITUDE, LONGITUDE, TUITIONFEE_IN, TUITIONFEE_OUT,
                                            cast(NPT41_PUB as int) NPT41_PUB, cast(NPT42_PUB as int) NPT42_PUB, cast(NPT43_PUB as int) NPT43_PUB, 
                                            cast(NPT44_PUB as int) NPT44_PUB, cast(NPT45_PUB as int) NPT45_PUB, 
                                            cast(NPT41_PRIV as int) NPT41_PRIV, cast(NPT42_PRIV as int) NPT42_PRIV, cast(NPT43_PRIV as int) NPT43_PRIV, 
                                            cast(NPT44_PRIV as int) NPT44_PRIV, cast(NPT45_PRIV as int) NPT45_PRIV, MD_EARN_WNE_P10,
                                            cast(SATMT25 as int) SATMT25, cast(SATMT75 as int) SATMT75, cast(SATVR25 as int) SATVR25, 
                                            cast(SATVR75 as int) SATVR75, cast(ACTCM25 as int) ACTCM25, cast(ACTCM75 as int) ACTCM75,
                                            ADM_RATE, C150_4, C150_L4, UGDS, CONTROL, LOCALE, RELAFFIL,
                                            HBCU, PBI, ANNHI, TRIBAL, AANAPII, HSI, NANTI, MENONLY, WOMENONLY
                                     FROM inst""", conn)
        elif name == 'field':
            with sqlite3.connect(db_path) as conn:
                tbl = pd.read_sql("SELECT UNITID, CREDLEV, CIPDESC, EARN_NE_MDN_3YR FROM fields", conn)
        elif name == 'geo':
            with sqlite3.connect(db_path) as conn:
                tbl = pd.read_sql("SELECT * FROM geo",conn)
                
        return tbl

In [87]:
#Tables on db
with sqlite3.connect(db_path) as conn:
    tbl = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",conn)
tbl

,name
0,fields
1,geo
2,user
3,relig
4,inst


Part 1. Establishing Database

In [56]:
conn = sqlite3.connect("cs.db")

In [57]:
cur = conn.cursor()

In [58]:
#Tables on db
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",conn)

,name
0,fields
1,geo
2,inst
3,user
4,relig


# Read data samples

In [157]:
with open("sample1.pickle", "rb") as file:
    sample1 = pickle.load(file)
with open("sample2.pickle", "rb") as file:
    sample2 = pickle.load(file)
with open("sample3.pickle", "rb") as file:
    sample3 = pickle.load(file)
# with open("sample4.pickle", "rb") as file:
#     sample4 = pickle.load(file)
with open("sample5.pickle", "rb") as file:
    sample5 = pickle.load(file)

In [60]:
#inst = pd.read_sql("SELECT * FROM inst",conn)

In [61]:
#conn.close()

In [62]:
sample1

{'user': 'mkjones',
 'limit_match': '6',
 'degree': {'pref': 10, 'val': '5', 'multi': 'N'},
 'sat_math': {'pref': 10, 'val': '650', 'multi': 'N'},
 'sat_cr': {'pref': 10, 'val': '720', 'multi': 'N'},
 'act': {'pref': 10, 'val': '0', 'multi': 'N'},
 'states': {'pref': 10,
  'val': ['MH',
   'MI',
   'MN',
   'MO',
   'MP',
   'MS',
   'MT',
   'NC',
   'ND',
   'NE',
   'NH',
   'NJ',
   'NM',
   'NV',
   'NY',
   'OH',
   'OK',
   'OR',
   'PA',
   'PR',
   'PW',
   'RI',
   'SC',
   'SD',
   'TN'],
  'multi': 'Y'},
 'input_zip': {'pref': '9', 'val': ['30315', '40'], 'multi': 'N'},
 'field': {'pref': '8', 'val': 'Computational Science.', 'multi': 'N'},
 'cost': {'pref': '5', 'val': ['10000', '5'], 'multi': 'N'},
 'salary': {'pref': '7', 'val': '120000', 'multi': 'N'},
 'ar': {'pref': '1', 'val': '30', 'multi': 'N'},
 'gr': {'pref': '3', 'val': '80', 'multi': 'N'},
 'sizes': {'pref': '10', 'val': ['Medium', 'Large'], 'multi': 'Y'},
 'types': {'pref': '9', 'val': ['Public', 'Private Nonp

In [63]:
sample2

{'user': 'mkjones',
 'limit_match': '2',
 'degree': {'pref': 10, 'val': '5', 'multi': 'N'},
 'sat_math': {'pref': 10, 'val': '0', 'multi': 'N'},
 'sat_cr': {'pref': 10, 'val': '0', 'multi': 'N'},
 'act': {'pref': 10, 'val': '27', 'multi': 'N'},
 'states': {'pref': 10, 'val': [''], 'multi': 'Y'},
 'input_zip': {'pref': '9', 'val': ['30315', '100'], 'multi': 'N'},
 'field': {'pref': '8', 'val': 'Computational Science.', 'multi': 'N'},
 'cost': {'pref': '5', 'val': ['10000', '5'], 'multi': 'N'},
 'salary': {'pref': '10', 'val': '0', 'multi': 'N'},
 'ar': {'pref': '1', 'val': '30', 'multi': 'N'},
 'gr': {'pref': '3', 'val': '80', 'multi': 'N'},
 'sizes': {'pref': '10', 'val': ['Medium', 'Large'], 'multi': 'Y'},
 'types': {'pref': '9', 'val': ['Public'], 'multi': 'Y'},
 'urban': {'pref': '10', 'val': ['City'], 'multi': 'Y'},
 'missions': {'pref': '10',
  'val': ['Men-Only College', 'Historically Black College and University'],
  'multi': 'Y'},
 'religs': {'pref': '10',
  'val': ['Free Metho

In [64]:
sample3

{'user': 'mkjones',
 'limit_match': '6',
 'degree': {'pref': 10, 'val': '3', 'multi': 'N'},
 'sat_math': {'pref': 10, 'val': '0', 'multi': 'N'},
 'sat_cr': {'pref': 10, 'val': '0', 'multi': 'N'},
 'act': {'pref': 10, 'val': '0', 'multi': 'N'},
 'states': {'pref': 10, 'val': [''], 'multi': 'Y'},
 'input_zip': {'pref': '10', 'val': ['30315', '0'], 'multi': 'N'},
 'field': {'pref': '10', 'val': '', 'multi': 'N'},
 'cost': {'pref': '10', 'val': ['0', '3'], 'multi': 'N'},
 'salary': {'pref': '10', 'val': '0', 'multi': 'N'},
 'ar': {'pref': '10', 'val': '0', 'multi': 'N'},
 'gr': {'pref': '10', 'val': '0', 'multi': 'N'},
 'sizes': {'pref': '10', 'val': [''], 'multi': 'Y'},
 'types': {'pref': '10', 'val': [''], 'multi': 'Y'},
 'urban': {'pref': '10', 'val': [''], 'multi': 'Y'},
 'missions': {'pref': '10', 'val': [''], 'multi': 'Y'},
 'religs': {'pref': '10', 'val': [''], 'multi': 'Y'}}

In [158]:
sample5

{'user': 'mkjones',
 'limit_match': '6',
 'degree': {'pref': 10, 'val': '3', 'multi': 'N'},
 'sat_math': {'pref': 10, 'val': '500', 'multi': 'N'},
 'sat_cr': {'pref': 10, 'val': '500', 'multi': 'N'},
 'act': {'pref': 10, 'val': '20', 'multi': 'N'},
 'states': {'pref': 10, 'val': ['FL', 'GA'], 'multi': 'Y'},
 'input_zip': {'pref': '10', 'val': ['30315', '400'], 'multi': 'N'},
 'field': {'pref': '10', 'val': 'Computational Science.', 'multi': 'N'},
 'cost': {'pref': '10', 'val': ['30000', '3'], 'multi': 'N'},
 'salary': {'pref': '10', 'val': '100000', 'multi': 'N'},
 'ar': {'pref': '10', 'val': '31', 'multi': 'N'},
 'gr': {'pref': '10', 'val': '41', 'multi': 'N'},
 'types': {'pref': '10', 'val': 'Public', 'multi': 'N'},
 'sizes': {'pref': '10', 'val': ['Medium', 'Large'], 'multi': 'Y'},
 'urban': {'pref': '10', 'val': ['City', 'Suburban'], 'multi': 'Y'},
 'missions': {'pref': '10',
  'val': ['Men-Only College', 'Historically Black College and University'],
  'multi': 'Y'},
 'religs': {'p

# Get Keys (features)

In [65]:
len(sample1)

18

In [66]:
list(sample1.keys())

['user',
 'limit_match',
 'degree',
 'sat_math',
 'sat_cr',
 'act',
 'states',
 'input_zip',
 'field',
 'cost',
 'salary',
 'ar',
 'gr',
 'sizes',
 'types',
 'urban',
 'missions',
 'religs']

In [246]:
data = sample5

In [247]:
def determine_constraint_type(data_dict):
    hard_list = ['degree','sat_math','sat_cr','act','states'] #these features are always hard
    soft_list = ['input_zip','field','cost','salary','ar','gr','sizes','types','urban','missions','religs'] #these features can be hard if user sets them at 10. Need to check 
    full_list = ['degree','sat_math','sat_cr','act','states','input_zip','field','cost','salary','ar','gr','sizes','types','urban','missions','religs']
    # Create empty list to store features to be removed from soft_list
    to_remove = []
    for feat in soft_list:
        #Determine which soft contraints to add to hard list if any
        if (int(data_dict[feat]['pref']) == 10) and ((data_dict[feat]['val'] != ['']) or (data_dict[feat]['val'] != '') or (data_dict[feat]['val'] != '0')):
            hard_list.append(feat)
            to_remove.append(feat)
        #Determine which soft contraints to remove if no input was provided
        if (data_dict[feat]['val'] == ['']) or (data_dict[feat]['val'] == '') or (data_dict[feat]['val'] == '0'):
            if feat not in to_remove:
                to_remove.append(feat)
        if feat == 'cost':
            if (data_dict[feat]['val'][0] == '0') and (feat not in to_remove):
                to_remove.append(feat)
        if feat == 'input_zip':
            if (data_dict[feat]['val'][1] == '0') and (feat not in to_remove):
                to_remove.append(feat)

    # Remove features from soft_list that were added to to_remove list
    for feat in to_remove:
        soft_list.remove(feat)

    to_remove = []
    for feat in hard_list:
        #Determine which hard contraints to remove if no input was provided
        if (data_dict[feat]['val'] == ['']) or (data_dict[feat]['val'] == '') or (data_dict[feat]['val'] == '0'):
            to_remove.append(feat)
        if feat == 'cost':
            if data_dict[feat]['val'][0] == '0':
                to_remove.append(feat)
        if feat == 'input_zip':
            if data_dict[feat]['val'][1] == '0':
                to_remove.append(feat)

    for feat in to_remove:
        hard_list.remove(feat)
    
    return full_list, hard_list, soft_list

In [248]:
full_list, hard_list, soft_list = determine_constraint_type(data)
print(hard_list)
print(soft_list)

['degree', 'sat_math', 'sat_cr', 'act', 'states', 'input_zip', 'field', 'cost', 'salary', 'ar', 'gr', 'sizes', 'types', 'urban', 'missions', 'religs']
[]


# Add Missing Columns to inst table

In [82]:
# tuition = pd.read_excel('tuition.xlsx')
# tuition.shape

(6681, 3)

In [81]:
# inst = pd.read_sql("SELECT * FROM inst", conn)
# inst.shape

(6681, 174)

In [83]:
# inst2 = sqldf("""SELECT i.*, TUITIONFEE_IN, TUITIONFEE_OUT
#                   FROM inst i
#                   JOIN tuition t
#                   ON i.UNITID = t.UNITID""")
# inst2.shape

(6681, 176)

In [84]:
# inst2.to_sql('inst',conn,if_exists='replace',index=False)

In [106]:
# rates = pd.read_excel('add_grad_rates.xlsx')
# rates.shape

(6681, 4)

In [107]:
# inst = pd.read_sql("SELECT * FROM inst", conn)
# inst.shape

(6681, 176)

In [108]:
# inst3 = sqldf("""SELECT i.*, ADM_RATE, C150_4, C150_L4
#                   FROM inst i
#                   JOIN rates t
#                   ON i.UNITID = t.UNITID""")
# inst3.shape

(6681, 179)

In [109]:
# inst3.to_sql('inst',conn,if_exists='replace',index=False)

In [122]:
hard_list

['degree',
 'sat_math',
 'sat_cr',
 'act',
 'states',
 'input_zip',
 'field',
 'cost',
 'salary',
 'ar',
 'gr',
 'sizes',
 'types',
 'urban',
 'missions',
 'religs']

# calc_dist()

In [136]:
def calc_dist(input_zip):
    geo = DatabaseHandler.get_table('geo')
    inst = DatabaseHandler.get_table('inst')[['UNITID','LATITUDE','LONGITUDE']]
    # Get lat lon of input zip
    lat =float(list(geo[geo['zip']==input_zip]['lat'])[0])
    lon =float(list(geo[geo['zip']==input_zip]['lng'])[0])
    # Get distance between each inst and input zip
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    return inst.loc[:, ['UNITID']].assign(DISTANCE_MI=idx)

# calc_cost()

# apply_hard_constraints()

In [156]:
data

{'user': 'mkjones',
 'limit_match': '6',
 'degree': {'pref': 10, 'val': '3', 'multi': 'N'},
 'sat_math': {'pref': 10, 'val': '500', 'multi': 'N'},
 'sat_cr': {'pref': 10, 'val': '500', 'multi': 'N'},
 'act': {'pref': 10, 'val': '20', 'multi': 'N'},
 'states': {'pref': 10, 'val': ['FL', 'GA'], 'multi': 'Y'},
 'input_zip': {'pref': '10', 'val': ['30315', '400'], 'multi': 'N'},
 'field': {'pref': '10', 'val': 'Computer Science.', 'multi': 'N'},
 'cost': {'pref': '10', 'val': ['30000', '3'], 'multi': 'N'},
 'salary': {'pref': '10', 'val': '100000', 'multi': 'N'},
 'ar': {'pref': '10', 'val': '38', 'multi': 'N'},
 'gr': {'pref': '10', 'val': '41', 'multi': 'N'},
 'types': {'pref': '10', 'val': ['Public'], 'multi': 'N'},
 'sizes': {'pref': '10', 'val': ['Medium', 'Large'], 'multi': 'Y'},
 'urban': {'pref': '10', 'val': ['City', 'Suburban'], 'multi': 'Y'},
 'missions': {'pref': '10',
  'val': ['Men-Only College', 'Historically Black College and University'],
  'multi': 'Y'},
 'religs': {'pref

In [241]:
sample5

{'user': 'mkjones',
 'limit_match': '6',
 'degree': {'pref': 10, 'val': '3', 'multi': 'N'},
 'sat_math': {'pref': 10, 'val': '500', 'multi': 'N'},
 'sat_cr': {'pref': 10, 'val': '500', 'multi': 'N'},
 'act': {'pref': 10, 'val': '20', 'multi': 'N'},
 'states': {'pref': 10, 'val': ['FL', 'GA'], 'multi': 'Y'},
 'input_zip': {'pref': '10', 'val': ['30315', '400'], 'multi': 'N'},
 'field': {'pref': '10', 'val': 'Computational Science.', 'multi': 'N'},
 'cost': {'pref': '10', 'val': ['30000', '3'], 'multi': 'N'},
 'salary': {'pref': '10', 'val': '100000', 'multi': 'N'},
 'ar': {'pref': '10', 'val': '31', 'multi': 'N'},
 'gr': {'pref': '10', 'val': '41', 'multi': 'N'},
 'types': {'pref': '10', 'val': 'Public', 'multi': 'N'},
 'sizes': {'pref': '10', 'val': ['Medium', 'Large'], 'multi': 'Y'},
 'urban': {'pref': '10', 'val': ['City', 'Suburban'], 'multi': 'Y'},
 'missions': {'pref': '10',
  'val': ['Men-Only College', 'Historically Black College and University'],
  'multi': 'Y'},
 'religs': {'p

In [162]:
data['types']['val']

'Public'

In [244]:
def apply_constraints(data_dict, full_list):
    filtered = DatabaseHandler.get_table('inst')
    for constraint in full_list:
        if constraint == 'input_zip':
            if data_dict[constraint]['val'][1] != 0: #Ignore constraint if no max distance specified
                inst_dist = calc_dist(data_dict[constraint]['val'][0])
                in_max_dist = inst_dist[inst_dist['DISTANCE_MI'] <= int(data_dict[constraint]['val'][1])]
                filtered = filtered.merge(in_max_dist, on='UNITID')
        elif constraint == 'states':
            if data_dict[constraint]['val'] != ['']: #Ignore constraint if no states specified
                filtered = filtered[filtered['STABBR'].isin(data_dict[constraint]['val'])]
        elif constraint == 'cost':
            if data_dict[constraint]['val'][0] != 0: #Ignore constraint if no max cost specified
                if data['types']['val'] == 'Public':
                    if data_dict[constraint]['val'][1] == '1': #if HI 0-$30K
                        filtered = filtered.rename(columns={'NPT41_PUB': 'cost'}) # To identify the column to display
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    elif data_dict[constraint]['val'][1] == '2':  #if HI >$30K <=$48K
                        filtered = filtered.rename(columns={'NPT42_PUB': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    elif data_dict[constraint]['val'][1] == '2':  #if HI >$48K <=$75K
                        filtered = filtered.rename(columns={'NPT43_PUB': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    elif data_dict[constraint]['val'][1] == '2':  #if HI >$75K <=$110K
                        filtered = filtered.rename(columns={'NPT44_PUB': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    else: #HI >$110k
                        filtered = filtered.rename(columns={'NPT45_PUB': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                else: #Private
                    if data_dict[constraint]['val'][1] == '1': #if HI 0-$30K
                        filtered = filtered.rename(columns={'NPT41_PRIV': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    elif data_dict[constraint]['val'][1] == '2':  #if HI >$30K <=$48K
                        filtered = filtered.rename(columns={'NPT42_PRIV': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    elif data_dict[constraint]['val'][1] == '2':  #if HI >$48K <=$75K
                        filtered = filtered.rename(columns={'NPT43_PRIV': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    elif data_dict[constraint]['val'][1] == '2':  #if HI >$75K <=$110K
                        filtered = filtered.rename(columns={'NPT44_PRIV': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
                    else: #HI >$110k
                        filtered = filtered.rename(columns={'NPT45_PRIV': 'cost'})
                        filtered = filtered[filtered['cost'] <= int(data_dict[constraint]['val'][0])]
        elif constraint =='field':
            if data_dict[constraint]['val'] != '': #Ignore filter on Field if none specified by user
                #First merge on degree/field constraint to get valid UNITIDs or universities that offer major 
                fields_tbl = DatabaseHandler.get_table('field')[['UNITID','CIPDESC']].drop_duplicates()
                offered = fields_tbl[fields_tbl['CIPDESC'] == data_dict[constraint]['val']]
                filtered = filtered.merge(offered, on='UNITID')
        elif constraint =='degree':
            #First merge on degree/field constraint to get valid UNITIDs or universities that offer degree 
            degree_tbl = DatabaseHandler.get_table('field')[['UNITID','CREDLEV']].drop_duplicates()
            offered = degree_tbl[degree_tbl['CREDLEV'] == data_dict[constraint]['val']]
            filtered = filtered.merge(offered, on='UNITID')
        
    return filtered

In [315]:
def apply_constraints(data_dict, full_list):
    filtered = DatabaseHandler.get_table('inst')
    for constraint in full_list:
        if constraint == 'input_zip':
            if data_dict[constraint]['val'][1] != 0: 
                inst_dist = calc_dist(data_dict[constraint]['val'][0])
                filtered = filtered.merge(inst_dist, on='UNITID')
                filtered['input_zip'] = (filtered['DISTANCE_MI'] <= int(data_dict[constraint]['val'][1])).astype(int)
            else:
                filtered['input_zip'] = 1
                filtered['DISTANCE_MI'] = None
        elif constraint == 'states':
            if data_dict[constraint]['val'] != ['']: 
                filtered['states'] = (filtered['STABBR'].isin(data_dict[constraint]['val'])).astype(int)
            else:
                filtered['states'] = 1
        elif constraint == 'cost':
            if data['types']['val'] == 'Public':
                if data_dict[constraint]['val'][1] == '1': #if HI 0-$30K
                    filtered = filtered.rename(columns={'NPT41_PUB': 'NPT'}) # To identify the column to display
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['NPT'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1
                elif data_dict[constraint]['val'][1] == '2':  #if HI >$30K <=$48K
                    filtered = filtered.rename(columns={'NPT42_PUB': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['NPT'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1
                elif data_dict[constraint]['val'][1] == '3':  #if HI >$48K <=$75K
                    filtered = filtered.rename(columns={'NPT43_PUB': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['NPT'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1
                elif data_dict[constraint]['val'][1] == '4':  #if HI >$75K <=$110K
                    filtered = filtered.rename(columns={'NPT44_PUB': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['cost'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1   
                else: #HI >$110k
                    filtered = filtered.rename(columns={'NPT45_PUB': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['cost'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1 
            else: #Private
                if data_dict[constraint]['val'][1] == '1': #if HI 0-$30K
                    filtered = filtered.rename(columns={'NPT41_PRIV': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['cost'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1 
                elif data_dict[constraint]['val'][1] == '2':  #if HI >$30K <=$48K
                    filtered = filtered.rename(columns={'NPT42_PRIV': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['cost'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1
                elif data_dict[constraint]['val'][1] == '3':  #if HI >$48K <=$75K
                    filtered = filtered.rename(columns={'NPT43_PRIV': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['cost'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1
                elif data_dict[constraint]['val'][1] == '4':  #if HI >$75K <=$110K
                    filtered = filtered.rename(columns={'NPT44_PRIV': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['cost'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1
                else: #HI >$110k
                    filtered = filtered.rename(columns={'NPT45_PRIV': 'NPT'})
                    if data_dict[constraint]['val'][0] != 0:
                        filtered['cost'] = (filtered['cost'] <= int(data_dict[constraint]['val'][0])).astype(int)
                    else:
                        filtered['cost'] = 1
        elif constraint =='field':
            if data_dict[constraint]['val'] != '': #Ignore filter on Field if none specified by user
                #First merge on degree/field constraint to get valid UNITIDs or universities that offer major 
                fields_tbl = DatabaseHandler.get_table('field')[['UNITID','CIPDESC']].drop_duplicates()
                offered = fields_tbl[fields_tbl['CIPDESC'] == data_dict[constraint]['val']]
                filtered['field'] = filtered['UNITID'].isin(offered['UNITID']).astype(int) 
            else:
                filtered['field'] = 1
        elif constraint =='degree':
            #First merge on degree/field constraint to get valid UNITIDs or universities that offer degree 
            degree_tbl = DatabaseHandler.get_table('field')[['UNITID','CREDLEV']].drop_duplicates()
            offered = degree_tbl[degree_tbl['CREDLEV'] == data_dict[constraint]['val']]
            filtered['degree'] = filtered['UNITID'].isin(offered['UNITID']).astype(int)
        elif constraint =='sat_math':
            filtered['sat_math'] = (filtered['SATMT25'] <= int(data_dict[constraint]['val'])).astype(int)  
    columns = ['UNITID','INSTNM','ZIP','CITY','STABBR','LATITUDE','LONGITUDE','NPT','DISTANCE_MI',
               'SATMT25', 'SATMT75', 'SATVR25', 'SATVR75', 'ACTCM25', 'ACTCM75',
               'input_zip','states','cost','field','degree','sat_math']
    return filtered[columns]

In [275]:
# list(test.columns)

In [264]:
sample5

{'user': 'mkjones',
 'limit_match': '6',
 'degree': {'pref': 10, 'val': '3', 'multi': 'N'},
 'sat_math': {'pref': 10, 'val': '500', 'multi': 'N'},
 'sat_cr': {'pref': 10, 'val': '500', 'multi': 'N'},
 'act': {'pref': 10, 'val': '20', 'multi': 'N'},
 'states': {'pref': 10, 'val': ['FL', 'GA'], 'multi': 'Y'},
 'input_zip': {'pref': '10', 'val': ['30315', '400'], 'multi': 'N'},
 'field': {'pref': '10', 'val': 'Computational Science.', 'multi': 'N'},
 'cost': {'pref': '10', 'val': ['30000', '3'], 'multi': 'N'},
 'salary': {'pref': '10', 'val': '100000', 'multi': 'N'},
 'ar': {'pref': '10', 'val': '31', 'multi': 'N'},
 'gr': {'pref': '10', 'val': '41', 'multi': 'N'},
 'types': {'pref': '10', 'val': 'Public', 'multi': 'N'},
 'sizes': {'pref': '10', 'val': ['Medium', 'Large'], 'multi': 'Y'},
 'urban': {'pref': '10', 'val': ['City', 'Suburban'], 'multi': 'Y'},
 'missions': {'pref': '10',
  'val': ['Men-Only College', 'Historically Black College and University'],
  'multi': 'Y'},
 'religs': {'p

In [316]:
test =  apply_constraints(data,full_list)
test.shape

(6681, 21)

In [319]:
pd.read_sql("SELECT * FROM inst WHERE SATMT25 is null",conn) 

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,INSTURL,NPCURL,HCM2,...,OMENRYP_PELL_PT_POOLED_SUPP,OMENRAP_PELL_PT_POOLED_SUPP,OMAWDP8_PELL_PT_POOLED_SUPP,OMENRUP_PELL_PT_POOLED_SUPP,GT_THRESHOLD_P6_SUPP,TUITIONFEE_IN,TUITIONFEE_OUT,ADM_RATE,C150_4,C150_L4
0,100690,02503400,025034,Amridge University,Montgomery,AL,36117-3553,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,0,...,0,0.186,0.4186,0.3953,0.5132,6950.0,6950.0,NaN,0.2500,NaN
1,100760,00100700,001007,Central Alabama Community College,Alexander City,AL,35010,www.cacc.edu/,https://www.cacc.edu/net-price-calculator/,0,...,0.0112,0.3073,0.0978,0.5838,0.375,4930.0,8860.0,NaN,NaN,0.1920
2,100812,00100800,001008,Athens State University,Athens,AL,35611,www.athens.edu/,www.athens.edu/financial-aid/net-price-calcula...,0,...,0.0181,0.1377,0.5192,0.3251,0.6619,NaN,NaN,NaN,NaN,NaN
3,101028,01218200,012182,Chattahoochee Valley Community College,Phenix City,AL,36869,www.cv.edu/,external.cv.edu/npc/index.html,0,...,0.0305,0.2267,0.2301,0.5127,0.4078,4920.0,8910.0,NaN,NaN,0.2246
4,101116,01303906,013039,South University-Montgomery,Montgomery,AL,36116,www.southuniversity.edu/,www.southuniversity.edu/montgomery/net-price-c...,1,...,0,0.3519,0.1185,0.5296,0.3982,17014.0,17014.0,NaN,0.1429,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5591,49576719,00332917,003329,Pennsylvania State University-Penn State Wilke...,Lehman,PA,18627-0217,wilkesbarre.psu.edu/,None,0,...,None,None,None,None,None,13604.0,22196.0,NaN,NaN,NaN
5592,49576720,00332918,003329,Pennsylvania State University-Penn State York,York,PA,17403-3326,york.psu.edu/,None,0,...,None,None,None,None,None,14486.0,23476.0,NaN,NaN,NaN
5593,49576721,00332919,003329,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,greatvalley.psu.edu/,None,0,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
5594,49576722,00332920,003329,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,harrisburg.psu.edu/,None,0,...,None,None,None,None,None,15216.0,24926.0,NaN,NaN,NaN


In [318]:
sqldf("SELECT * FROM test WHERE sat_math =0 and SATMT25 is null")

,UNITID,INSTNM,ZIP,CITY,STABBR,LATITUDE,LONGITUDE,NPT,DISTANCE_MI,SATMT25,...,SATVR25,SATVR75,ACTCM25,ACTCM75,input_zip,states,cost,field,degree,sat_math
0,100690,Amridge University,36117-3553,Montgomery,AL,32.362609,-86.17401,NaN,139.080645,None,...,NaN,NaN,NaN,NaN,1,0,0,0,1,0
1,100760,Central Alabama Community College,35010,Alexander City,AL,32.92478,-85.945266,4201.0,105.030607,None,...,NaN,NaN,NaN,NaN,1,0,1,0,0,0
2,100812,Athens State University,35611,Athens,AL,34.806793,-86.964698,NaN,165.996024,None,...,NaN,NaN,NaN,NaN,1,0,0,0,1,0
3,101028,Chattahoochee Valley Community College,36869,Phenix City,AL,32.42391,-85.031485,4402.0,96.036572,None,...,NaN,NaN,NaN,NaN,1,0,1,0,0,0
4,101116,South University-Montgomery,36116,Montgomery,AL,32.342684,-86.216488,NaN,141.843030,None,...,NaN,NaN,NaN,NaN,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5591,49576719,Pennsylvania State University-Penn State Wilke...,18627-0217,Lehman,PA,41.31559,-76.02235,NaN,696.862958,None,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5592,49576720,Pennsylvania State University-Penn State York,17403-3326,York,PA,39.92142,-76.71277,NaN,603.350253,None,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5593,49576721,Pennsylvania State University-Penn State Great...,19355-1488,Malvern,PA,40.0466,-75.5302,NaN,656.257973,None,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5594,49576722,Pennsylvania State University-Penn State Harri...,17057-4846,Middletown,PA,40.19437,-76.72659,NaN,615.843200,None,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [277]:
test

,UNITID,INSTNM,ZIP,CITY,STABBR,LATITUDE,LONGITUDE,NPT,DISTANCE_MI,input_zip,states,cost
0,100654,Alabama A & M University,35762,Normal,AL,34.783368,-86.568502,17694.0,145.448313,1,0,1
1,100663,University of Alabama at Birmingham,35294-0110,Birmingham,AL,33.505697,-86.799345,17857.0,139.732231,1,0,1
2,100690,Amridge University,36117-3553,Montgomery,AL,32.362609,-86.17401,NaN,139.080645,1,0,0
3,100706,University of Alabama in Huntsville,35899,Huntsville,AL,34.724557,-86.640449,17054.0,147.036320,1,0,1
4,100724,Alabama State University,36104-0271,Montgomery,AL,32.364317,-86.295677,19853.0,144.337436,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6676,49576719,Pennsylvania State University-Penn State Wilke...,18627-0217,Lehman,PA,41.31559,-76.02235,NaN,NaN,0,0,0
6677,49576720,Pennsylvania State University-Penn State York,17403-3326,York,PA,39.92142,-76.71277,NaN,NaN,0,0,0
6678,49576721,Pennsylvania State University-Penn State Great...,19355-1488,Malvern,PA,40.0466,-75.5302,NaN,NaN,0,0,0
6679,49576722,Pennsylvania State University-Penn State Harri...,17057-4846,Middletown,PA,40.19437,-76.72659,NaN,NaN,0,0,0


In [262]:
sqldf("SELECT * FROM test WHERE states =1")

,UNITID,INSTNM,ZIP,CITY,STABBR,LATITUDE,LONGITUDE,TUITIONFEE_IN,TUITIONFEE_OUT,NPT41_PUB,...,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,states,DISTANCE_MI,input_zip
0,132374,Atlantic Technical College,33063-3902,Coconut Creek,FL,26.24278,-80.192271,NaN,NaN,9158.0,...,0,0,0,0,0,0,0,1,NaN,0
1,132408,The Baptist College of Florida,32440,Graceville,FL,30.962708,-85.503931,12150.0,12150.0,NaN,...,0,0,0,0,0,0,0,1,200.335035,1
2,132471,Barry University,33161-6695,Miami,FL,25.878908,-80.198925,30940.0,30940.0,NaN,...,0,0,0,1,0,0,0,1,NaN,0
3,132602,Bethune-Cookman University,32114-3099,Daytona Beach,FL,29.207195,-81.032946,14794.0,14794.0,NaN,...,0,0,0,0,0,0,0,1,368.010768,1
4,132657,Lynn University,33431-5598,Boca Raton,FL,26.385985,-80.122806,39350.0,39350.0,NaN,...,0,0,0,0,0,0,0,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,48511112,Georgia Military College - Dublin,31021-5149,Dublin,GA,32.49002,-82.93435,6210.0,6210.0,NaN,...,None,None,None,None,None,None,None,1,118.545861,1
565,48511113,Georgia Military College - Eastman,31023-6109,Eastman,GA,32.18288,-83.1991,6210.0,6210.0,NaN,...,None,None,None,None,None,None,None,1,125.469724,1
566,49005401,HCI College - Fort Lauderdale Campus,33309,Fort Lauderdale,FL,26.18793,-80.17385,17670.0,17670.0,NaN,...,None,None,None,None,None,None,None,1,NaN,0
567,49022101,MAK Beauty Institute - Duluth,30096-4846,Duluth,GA,33.97691,-84.14791,NaN,NaN,NaN,...,None,None,None,None,None,None,None,1,23.225909,1


In [226]:
test2 = apply_hard_constraints(data,hard_list)
test2.shape

(238, 43)

In [237]:
test3 = apply_hard_constraints(data,hard_list)
test3.shape

(8, 44)

In [239]:
test4 = apply_hard_constraints(data,hard_list)
test4.shape

(8, 45)

In [245]:
test5 = apply_hard_constraints(data,hard_list)
test5.shape

(3, 45)

In [211]:
test2

,UNITID,INSTNM,ZIP,CITY,STABBR,LATITUDE,LONGITUDE,TUITIONFEE_IN,TUITIONFEE_OUT,NPT41_PUB,...,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,DISTANCE_MI
0,100654,Alabama A & M University,35762,Normal,AL,34.783368,-86.568502,10024.0,18634.0,14694.0,...,1,0,0,0,0,0,0,0,0,145.448313
1,100663,University of Alabama at Birmingham,35294-0110,Birmingham,AL,33.505697,-86.799345,8568.0,20400.0,13443.0,...,0,0,0,0,0,0,0,0,0,139.732231
3,100706,University of Alabama in Huntsville,35899,Huntsville,AL,34.724557,-86.640449,11338.0,23734.0,13631.0,...,0,0,0,0,0,0,0,0,0,147.036320
4,100724,Alabama State University,36104-0271,Montgomery,AL,32.364317,-86.295677,11068.0,19396.0,19581.0,...,1,0,0,0,0,0,0,0,0,144.337436
5,100751,The University of Alabama,35487-0100,Tuscaloosa,AL,33.211875,-87.545978,11620.0,31090.0,17523.0,...,0,0,0,0,0,0,0,0,0,185.467690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,483045,Central Georgia Technical College,31088,Warner Robins,GA,32.545341,-83.66765,3042.0,5442.0,1458.0,...,0,1,0,0,0,0,0,0,0,90.061867
923,484473,University of Florida-Online,32611,Gainesville,FL,29.649413,-82.339848,3876.0,16579.0,848.0,...,0,0,0,0,0,0,0,0,0,304.734742
925,484932,BridgeValley Community & Technical College,25136,South Charleston,WV,38.353846,-81.6983,5142.0,11140.0,7676.0,...,0,0,0,0,0,0,0,0,0,354.573212
928,485458,Coastal Pines Technical College,31503,Waycross,GA,31.212781,-82.3821,2759.0,4906.0,-97.0,...,0,0,0,0,0,0,0,0,0,207.859540


In [105]:
sqldf("SELECT * FROM test WHERE UNITID ='49576721' ")

,UNITID,INSTNM,ZIP,CITY,STABBR,TUITIONFEE_IN,TUITIONFEE_OUT,NPT41_PUB,NPT42_PUB,NPT43_PUB,...,RELAFFIL,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY
0,49576721,Pennsylvania State University-Penn State Great...,19355-1488,Malvern,PA,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Define Features and Filters

In [21]:
def features_dict(feature1, feature2, feature3):
    featuresDict = {}
    featuresDict['feature1']= feature1
    featuresDict['feature2']= feature2
    featuresDict['feature3']= feature3
    return featuresDict

In [22]:
featuresDict = features_dict('OMENRUP_PELL_NFT_POOLED_SUPP', 'OMENRYP_PELL_FT_POOLED_SUPP', 'OMENRAP_PELL_FT_POOLED_SUPP')

In [10]:
def filter_list(filter1, filter2):
    l = list()
    l.append(filter1)
    l.append(filter2)
    return l

In [11]:
filterList = filter_list('SATMT25','SATVR25')
filterList

['SATMT25', 'SATVR25']

In [12]:
def filter_values(filter1,filter1input, filter2, filter2input):
    filters_dict = {}
    filters_dict[filter1] = filter1input
    filters_dict[filter2] = filter2input
    return filters_dict

In [13]:
filterValues = filter_values('SATMT25', '586', 'SATVR25', '789')
filterValues

{'SATMT25': '586', 'SATVR25': '789'}

Part 2. User Inputs

Part 2a. Preferences:
Input provided: Importance from scale 1 to 10
Output created: Allocation Percentage

In [18]:
#a, b, c represents importance user provides for each preference
#must be in same order as they are called in normalized matrix
f1weight=4
f2weight=5
f3weight=6
raw_weights = np.array([f1weight, f2weight, f3weight])
sum_weights = np.sum(raw_weights)
perc_weights = raw_weights/sum_weights
perc_weights

array([0.26666667, 0.33333333, 0.4       ])

Part 2b. User ID Creation
Input provided: email
Output created: 1) user_id 2) inserts user_id and email into "user" table
Method: function (user_id_generator)

In [33]:
#Scenario 1 Testing
email = 'mkjones@gatech.edu'
user_id = int(hashlib.sha256(email.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID

In [34]:
user_id

9987952

In [35]:
#Scenario #2
#trying different email
email2 = 'michaelk.Jones@outlook.com'
user_id3 = int(hashlib.sha256(email2.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id3

80449489

In [36]:
#Scenario 3
#trying same email as scenario 2 but with all lowercase
email3 = 'michaelk.jones@outlook.com'
user_id4 = int(hashlib.sha256(email3.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id4

87012218

In [37]:
#Scenario 4
#same email as scenario 2 but added lower to have negate capitalization effects
email2 = 'michaelk.Jones@outlook.com'
user_id5 = int(hashlib.sha256(email2.lower().encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id5

87012218

In [38]:
def user_id_generator(email):
    id = int(hashlib.sha256(email.lower().encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
    return id

In [39]:
#test case of function
user_id_generator(email = 'test123@outlook.com')

83932234

In [40]:
email = 'test123@outlook.com'
user_id = user_id_generator(email = 'test123@outlook.com')

if pd.read_sql(f"SELECT * FROM user WHERE user_id ={user_id}",conn).empty:
    cur.execute(f"INSERT INTO user values('{email}',{user_id})")
else:
    print("Email exists")

In [42]:
pd.read_sql(f"SELECT * FROM user",conn)

,email,user_id
0,mkjones@gatech.edu,9987952
1,mykek.jones@gmail.com,95829733
2,michaelk.jones@outlook.com,87012218
3,test1@GMAIL.COM,65768313
4,test1@gmail.com,45591513


Part 2.c: Zipcode
user provided: zipcode
output created: temp dataframe of distance in miles per institution

In [9]:
geo = pd.read_sql("SELECT * FROM geo",conn)

In [16]:
geo.printSchema()

AttributeError: 'DataFrame' object has no attribute 'printSchema'

In [45]:
input_zip = "30315"

In [46]:
lat =float(list(geo[geo['zip']==input_zip]['lat'])[0])
lon =float(list(geo[geo['zip']==input_zip]['lng'])[0])
print(f"lat, lon: ({lat}, {lon})")

lat, lon: (33.70312, -84.38272)


In [51]:
inst[["UNITID","LATITUDE","LONGITUDE"]]

,UNITID,LATITUDE,LONGITUDE
0,100654,34.783368,-86.568502
1,100663,33.505697,-86.799345
2,100690,32.362609,-86.17401
3,100706,34.724557,-86.640449
4,100724,32.364317,-86.295677
...,...,...,...
6676,49576719,41.31559,-76.02235
6677,49576720,39.92142,-76.71277
6678,49576721,40.0466,-75.5302
6679,49576722,40.19437,-76.72659


In [57]:
## Function to use to find the distance of each university in miles
def inst_dist (lat,lon):
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    column_name = f"{user_id}_DISTANCE_MI"  # Adding user_id to the column title
    return inst.loc[:, ['UNITID']].assign(**{column_name: idx})

In [59]:
def inst_dist (lat,lon):
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    column_name = f"{user_id}_DISTANCE_MI"  # Adding user_id to the column title
    return inst.loc[:, ['UNITID']].assign(dist_miles = idx)

In [61]:
#Distances of each university to the input zip
dist_temp_table = inst_dist(lat,lon)
type(dist_temp_table)
dist_temp_table
inst = pd.merge(inst, dist_temp_table, on='UNITID')

In [5]:
pd.read_sql("SELECT * FROM inst",conn)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,INSTURL,NPCURL,HCM2,...,OMENRUP_PELL_NFT_POOLED_SUPP,OMENRYP_PELL_FT_POOLED_SUPP,OMENRAP_PELL_FT_POOLED_SUPP,OMAWDP8_PELL_FT_POOLED_SUPP,OMENRUP_PELL_FT_POOLED_SUPP,OMENRYP_PELL_PT_POOLED_SUPP,OMENRAP_PELL_PT_POOLED_SUPP,OMAWDP8_PELL_PT_POOLED_SUPP,OMENRUP_PELL_PT_POOLED_SUPP,GT_THRESHOLD_P6_SUPP
0,100654,00100200,001002,Alabama A & M University,Normal,AL,35762,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,0,...,0.2803,0.0116,0.3329,0.2958,0.3597,0.0056,0.3889,0.1833,0.4222,0.4606
1,100663,00105200,001052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,0,...,0.211,0.0149,0.2773,0.5243,0.1835,0.0211,0.277,0.4354,0.2665,0.659
2,100690,02503400,025034,Amridge University,Montgomery,AL,36117-3553,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,0,...,0.468,0,0.2798,0.2523,0.4679,0,0.186,0.4186,0.3953,0.5132
3,100706,00105500,001055,University of Alabama in Huntsville,Huntsville,AL,35899,www.uah.edu/,finaid.uah.edu/,0,...,0.187,0.0098,0.3154,0.4961,0.1787,0.0105,0.3632,0.3474,0.2789,0.717
4,100724,00100500,001005,Alabama State University,Montgomery,AL,36104-0271,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,0,...,0.2026,0.0097,0.3904,0.3061,0.2938,0.0778,0.4444,0.1111,0.3667,0.382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,49576719,00332917,003329,Pennsylvania State University-Penn State Wilke...,Lehman,PA,18627-0217,wilkesbarre.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None
6677,49576720,00332918,003329,Pennsylvania State University-Penn State York,York,PA,17403-3326,york.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None
6678,49576721,00332919,003329,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,greatvalley.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None
6679,49576722,00332920,003329,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,harrisburg.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None


In [119]:
def transform_zip_to_dist_col(zipcode):
    geo = pd.read_sql("SELECT * FROM geo",conn)
    inst = pd.read_sql("SELECT * FROM inst",conn)
    lat =float(list(geo[geo['zip']==zipcode]['lat'])[0])
    lon =float(list(geo[geo['zip']==zipcode]['lng'])[0])
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    distDF = inst.loc[:, ['UNITID']].assign(dist_miles = idx)
    inst = pd.merge(inst, distDF, on='UNITID')
    return inst

In [120]:
test = transform_zip_to_dist_col('30097')
test

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,INSTURL,NPCURL,HCM2,...,OMENRYP_PELL_FT_POOLED_SUPP,OMENRAP_PELL_FT_POOLED_SUPP,OMAWDP8_PELL_FT_POOLED_SUPP,OMENRUP_PELL_FT_POOLED_SUPP,OMENRYP_PELL_PT_POOLED_SUPP,OMENRAP_PELL_PT_POOLED_SUPP,OMAWDP8_PELL_PT_POOLED_SUPP,OMENRUP_PELL_PT_POOLED_SUPP,GT_THRESHOLD_P6_SUPP,dist_miles
0,100654,00100200,001002,Alabama A & M University,Normal,AL,35762,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,0,...,0.0116,0.3329,0.2958,0.3597,0.0056,0.3889,0.1833,0.4222,0.4606,147.593425
1,100663,00105200,001052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,0,...,0.0149,0.2773,0.5243,0.1835,0.0211,0.277,0.4354,0.2665,0.659,156.540810
2,100690,02503400,025034,Amridge University,Montgomery,AL,36117-3553,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,0,...,0,0.2798,0.2523,0.4679,0,0.186,0.4186,0.3953,0.5132,164.191256
3,100706,00105500,001055,University of Alabama in Huntsville,Huntsville,AL,35899,www.uah.edu/,finaid.uah.edu/,0,...,0.0098,0.3154,0.4961,0.1787,0.0105,0.3632,0.3474,0.2789,0.717,150.129854
4,100724,00100500,001005,Alabama State University,Montgomery,AL,36104-0271,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,0,...,0.0097,0.3904,0.3061,0.2938,0.0778,0.4444,0.1111,0.3667,0.382,169.201972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,49576719,00332917,003329,Pennsylvania State University-Penn State Wilke...,Lehman,PA,18627-0217,wilkesbarre.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,670.943595
6677,49576720,00332918,003329,Pennsylvania State University-Penn State York,York,PA,17403-3326,york.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,577.729643
6678,49576721,00332919,003329,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,greatvalley.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,630.970517
6679,49576722,00332920,003329,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,harrisburg.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,590.104574


Applying filters:

In [ ]:
.set_index('UNITID', inplace=True)

In [121]:
#decided to call table directly
def apply_filters_and_features(featuresDict, filterList, filterValues):
    filterquery = """
    SELECT
    UNITID,
    {0},
    {1},
    {2}
    FROM test
    WHERE {3} <= {4}
    AND {5} <= {6}
    """.format(featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'], filterList[0], filterValues[filterList[0]],filterList[1],  filterValues[filterList[1]])
    return sqldf(filterquery,globals())

In [122]:
appliedffs = apply_filters_and_features(featuresDict = featuresDict,filterList=filterList,filterValues=filterValues)

In [71]:
#stuck on getting argument df to work in format function
def apply_filters(df, featuresDict, filterList, filterValues):
    filterquery = """
    SELECT
    {1},
    {2},
    {3}
    FROM {0}
    WHERE {4} <= {5}
    AND {6} <= {7}
    """.format(df,featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'], filterList[0], filterValues[filterList[0]],filterList[1],  filterValues[filterList[1]])
    return sqldf(filterquery,locals())

In [85]:
#stuck on getting argument df to work in format function
def apply_filters(df, featuresDict, filterList, filterValues):
    filterquery = """
    SELECT
    {1},
    {2},
    {3}
    FROM {0}
    WHERE {4} <= {5}
    AND {6} <= {7}
    """.format(df, featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'], filterList[0], filterValues[filterList[0]],filterList[1],  filterValues[filterList[1]])
    return sqldf(filterquery,globals())

In [65]:
#stuck on getting argument df to work in format function
def apply_filters(df, featuresDict, filterList, filterValues):
    filterquery = f"""
    SELECT
    {featuresDict['feature1']},
    {featuresDict['feature2']},
    {featuresDict['feature3']}
    FROM {df}
    WHERE {filterList[0]} <= {filterValues[filterList[0]]}
    AND {filterList[1]} <= {filterValues[filterList[1]]}
    """
    return sqldf(filterquery,globals())

In [66]:
appliedfilters = apply_filters(df = test, featuresDict = featuresDict,filterList=filterList,filterValues=filterValues)
appliedfilters

PandaSQLException: (sqlite3.OperationalError) near "OPEID6": syntax error
[SQL: 
    SELECT
    OMENRUP_PELL_NFT_POOLED_SUPP,
    OMENRYP_PELL_FT_POOLED_SUPP,
    OMENRAP_PELL_FT_POOLED_SUPP
    FROM         UNITID     OPEID  OPEID6   
0       100654  00100200  001002  \
1       100663  00105200  001052   
2       100690  02503400  025034   
3       100706  00105500  001055   
4       100724  00100500  001005   
...        ...       ...     ...   
6676  49576719  00332917  003329   
6677  49576720  00332918  003329   
6678  49576721  00332919  003329   
6679  49576722  00332920  003329   
6680  49576723  00332921  003329   

                                                 INSTNM        CITY STABBR   
0                              Alabama A & M University      Normal     AL  \
1                   University of Alabama at Birmingham  Birmingham     AL   
2                                    Amridge University  Montgomery     AL   
3                   University of Alabama in Huntsville

Normalizing Features

part 1: find each feature's length

In [32]:
maxQuery = """
SELECT max({0}) x1FeatureLength, max({1}) x2FeatureLength, max({2}) x3FeatureLength
FROM appliedffs
where {0} not like 'PrivacySuppressed' and {1} not like 'PrivacySuppressed' and {2} not like 'PrivacySuppressed'
""".format(featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'])

In [45]:
featureLengths = sqldf(maxQuery,locals()).loc[0]
featureLengths

x1FeatureLength    0.8871
x2FeatureLength     0.241
x3FeatureLength    0.5972
Name: 0, dtype: object

In [46]:
x1FeatureLength = float(featureLengths['x1FeatureLength'])
x2FeatureLength = float(featureLengths['x2FeatureLength'])
x3FeatureLength = float(featureLengths['x3FeatureLength'])

In [47]:
x1FeatureLength

0.8871

part 2: applying length to normalize each feature

In [ ]:
normalizedQuery = """
SELECT
{0}/{3} feature1normalized,
{1}/{4} feature2normalized,
{2}/{5} feature3normalized
FROM inst
""".format(featuresDict['x1'],featuresDict['x2'],featuresDict['x3'], x1FeatureLength, x2FeatureLength, x3FeatureLength)

In [ ]:
normalizedDF = sqldf(normalizedQuery,locals())
normalizedDF

In [141]:
def normalize_features (features_Dict):
    #find length of each feature
    maxQuery = """
    SELECT max({0}) x1FeatureLength, max({1}) x2FeatureLength, max({2}) x3FeatureLength
    FROM appliedffs
    where {0} not like 'PrivacySuppressed' and {1} not like 'PrivacySuppressed' and {2} not like 'PrivacySuppressed'
    """.format(featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'])

    featureLengths = sqldf(maxQuery,globals()).loc[0]

    x1FeatureLength = float(featureLengths['x1FeatureLength'])
    x2FeatureLength = float(featureLengths['x2FeatureLength'])
    x3FeatureLength = float(featureLengths['x3FeatureLength'])

    #normalize each feature
    normalizedQuery = """
    SELECT
    UNITID,
    {0}/{3} feature1normalized,
    {1}/{4} feature2normalized,
    {2}/{5} feature3normalized
    FROM appliedffs
    """.format(featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'], x1FeatureLength, x2FeatureLength, x3FeatureLength)

    return sqldf(normalizedQuery,globals())

In [142]:
normalizedByUni = normalize_features(featuresDict)
#normalizedByUni.set_index('UNITID', inplace = True)

In [143]:
normalizedByUni

,UNITID,feature1normalized,feature2normalized,feature3normalized
0,100654,0.315973,0.048133,0.557435
1,100663,0.237854,0.061826,0.464334
2,100706,0.210799,0.040664,0.528131
3,100724,0.228385,0.040249,0.653717
4,100751,0.185548,0.036100,0.508540
...,...,...,...,...
928,487524,0.357682,0.007054,0.340589
929,488305,NaN,NaN,NaN
930,488785,NaN,NaN,NaN
931,490805,0.277083,0.048133,0.608841


Calculating Uni Rating

multiplying preferences to feature

In [169]:
def uni_rating_calc(df):
    normalizedMatrix = df.values
    ratingScore = np.dot(normalizedMatrix[:,1:4], perc_weights)
    return pd.DataFrame(np.c_[df.values[:, :1], ratingScore]).rename(columns = {0:'UNITID', 1:'Rating'})

In [170]:
test = uni_rating_calc(normalizedByUni)
test

,UNITID,Rating
0,100654,0.323278
1,100663,0.26977
2,100706,0.28102
3,100724,0.335806
4,100751,0.264929
...,...,...
928,487524,0.233969
929,488305,NaN
930,488785,NaN
931,490805,0.333469


In [162]:
normalizedByUni

,UNITID,feature1normalized,feature2normalized,feature3normalized
0,100654,0.315973,0.048133,0.557435
1,100663,0.237854,0.061826,0.464334
2,100706,0.210799,0.040664,0.528131
3,100724,0.228385,0.040249,0.653717
4,100751,0.185548,0.036100,0.508540
...,...,...,...,...
928,487524,0.357682,0.007054,0.340589
929,488305,NaN,NaN,NaN
930,488785,NaN,NaN,NaN
931,490805,0.277083,0.048133,0.608841


In [151]:
normalizedMatrix = normalizedByUni.values
normalizedMatrix[:,1:4]

array([[0.3159733964603765, 0.04813278008298755, 0.5574346952444742],
       [0.2378536805320708, 0.06182572614107884, 0.4643335565974548],
       [0.21079923345733287, 0.04066390041493776, 0.528131279303416],
       ...,
       [nan, nan, nan],
       [0.27708262879044077, 0.04813278008298755, 0.608841259209645],
       [0.19467929207530155, 0.03941908713692946, 0.37458137977227063]],
      dtype=object)

In [165]:
uniIndex = normalizedByUni.values[:, :1]

In [166]:
uniIndex

array([['100654'],
       ['100663'],
       ['100706'],
       ['100724'],
       ['100751'],
       ['100830'],
       ['100858'],
       ['100937'],
       ['101189'],
       ['101435'],
       ['101480'],
       ['101541'],
       ['101648'],
       ['101709'],
       ['102049'],
       ['102094'],
       ['102234'],
       ['102368'],
       ['102377'],
       ['102614'],
       ['104151'],
       ['104179'],
       ['104586'],
       ['105589'],
       ['105899'],
       ['106245'],
       ['106342'],
       ['106397'],
       ['106412'],
       ['106458'],
       ['106704'],
       ['106713'],
       ['107044'],
       ['107071'],
       ['107141'],
       ['107512'],
       ['107558'],
       ['107877'],
       ['110097'],
       ['110361'],
       ['110413'],
       ['110510'],
       ['110529'],
       ['110538'],
       ['110556'],
       ['110565'],
       ['110583'],
       ['110592'],
       ['110608'],
       ['110617'],
       ['110671'],
       ['110714'],
       ['111

In [140]:
test = np.c_[uniIndex, normalizedMatrix]
test

array([['100654', 0.3159733964603765, 0.04813278008298755,
        0.5574346952444742],
       ['100663', 0.2378536805320708, 0.06182572614107884,
        0.4643335565974548],
       ['100706', 0.21079923345733287, 0.04066390041493776,
        0.528131279303416],
       ...,
       ['488785', nan, nan, nan],
       ['490805', 0.27708262879044077, 0.04813278008298755,
        0.608841259209645],
       ['495767', 0.19467929207530155, 0.03941908713692946,
        0.37458137977227063]], dtype=object)

Output

In [178]:
def features_dict(feature1, feature2, feature3):
    featuresDict = {}
    featuresDict['feature1']= feature1
    featuresDict['feature2']= feature2
    featuresDict['feature3']= feature3
    return featuresDict

featuresDict = features_dict('OMENRUP_PELL_NFT_POOLED_SUPP', 'OMENRYP_PELL_FT_POOLED_SUPP', 'OMENRAP_PELL_FT_POOLED_SUPP')

def filter_list(filter1, filter2):
    l = list()
    l.append(filter1)
    l.append(filter2)
    return l

filterList = filter_list('SATMT25','SATVR25')

def filter_values(filter1,filter1input, filter2, filter2input):
    filters_dict = {}
    filters_dict[filter1] = filter1input
    filters_dict[filter2] = filter2input
    return filters_dict

filterValues = filter_values('SATMT25', '586', 'SATVR25', '789')

#Inputs from Users
f1weight=4
f2weight=5
f3weight=6
raw_weights = np.array([f1weight, f2weight, f3weight])
sum_weights = np.sum(raw_weights)
perc_weights = raw_weights/sum_weights
perc_weights

userzipcode = '30316'

def transform_zip_to_dist_col(zipcode):
    geo = pd.read_sql("SELECT * FROM geo",conn)
    inst = pd.read_sql("SELECT * FROM inst",conn)
    lat =float(list(geo[geo['zip']==zipcode]['lat'])[0])
    lon =float(list(geo[geo['zip']==zipcode]['lng'])[0])
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    distDF = inst.loc[:, ['UNITID']].assign(dist_miles = idx)
    inst = pd.merge(inst, distDF, on='UNITID')
    return inst

dist_appended = transform_zip_to_dist_col(zipcode = userzipcode)

def apply_filters_and_features(featuresDict, filterList, filterValues):
    filterquery = """
    SELECT
    UNITID,
    {0},
    {1},
    {2}
    FROM dist_appended
    WHERE {3} <= {4}
    AND {5} <= {6}
    """.format(featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'], filterList[0], filterValues[filterList[0]],filterList[1],  filterValues[filterList[1]])
    return sqldf(filterquery,globals())

appliedffs = apply_filters_and_features(featuresDict = featuresDict,filterList=filterList,filterValues=filterValues)

def normalize_features (features_Dict):
    #find length of each feature
    maxQuery = """
    SELECT max({0}) x1FeatureLength, max({1}) x2FeatureLength, max({2}) x3FeatureLength
    FROM appliedffs
    where {0} not like 'PrivacySuppressed' and {1} not like 'PrivacySuppressed' and {2} not like 'PrivacySuppressed'
    """.format(featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'])

    featureLengths = sqldf(maxQuery,globals()).loc[0]

    x1FeatureLength = float(featureLengths['x1FeatureLength'])
    x2FeatureLength = float(featureLengths['x2FeatureLength'])
    x3FeatureLength = float(featureLengths['x3FeatureLength'])

    #normalize each feature
    normalizedQuery = """
    SELECT
    UNITID,
    {0}/{3} feature1normalized,
    {1}/{4} feature2normalized,
    {2}/{5} feature3normalized
    FROM appliedffs
    """.format(featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'], x1FeatureLength, x2FeatureLength, x3FeatureLength)

    return sqldf(normalizedQuery,globals())

normalized_df = normalize_features(features_Dict = featuresDict)

def uni_rating_calc(df):
    normalizedMatrix = df.values
    ratingScore = np.dot(normalizedMatrix[:,1:4], perc_weights)
    return pd.DataFrame(np.c_[df.values[:, :1], ratingScore]).rename(columns = {0:'UNITID', 1:'Rating'})

finalusertable = uni_rating_calc(normalized_df)


In [181]:
finalusertable.to_csv('ratingsbyuni.csv', index=False)
finalusertable

,UNITID,Rating
0,100654,0.323278
1,100663,0.26977
2,100706,0.28102
3,100724,0.335806
4,100751,0.264929
...,...,...
928,487524,0.233969
929,488305,NaN
930,488785,NaN
931,490805,0.333469
